In [1]:
!pip install XlsxWriter

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import tensorflow as tf
import tensorflow.keras as keras
from keras import layers
import numpy as np
import os
import time
import xlsxwriter
import PIL
from IPython import display
import matplotlib.pyplot as plt

Unzip dataset file. Just need to do it one time.

In [ ]:
!unzip '/content/drive/MyDrive/img_align_celeba.zip' -d '/content/drive/MyDrive/train/'

Archive:  /content/drive/MyDrive/img_align_celeba.zip
replace /content/drive/MyDrive/train/001/001000.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Make models.
1. Generator: four deconvolution layers with 5x5 kernel, and strides for 2x2. They are all followed by a batch normalization and leakyReLU.
2. Discriminator: three convolution layers with 5x5 kernel, and strides for 2x2. They are followed by a leakyReLU. And, the last two convolution layers will dropout for 0.5

In [4]:
def make_generator_model():
  model = tf.keras.Sequential()
  model.add(layers.Dense((256 * 8 * 8), input_shape=(100,)))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())
  
  model.add(layers.Reshape((8, 8, 256)))
  assert model.output_shape == (None, 8, 8, 256)  # Note: None is the batch size

  model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  model.add(layers.Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', activation='tanh', use_bias=False))

  return model

def make_discriminator_model():
  model = tf.keras.Sequential()
  model.add(layers.Input(shape=(128, 128, 3)))
  model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
  model.add(layers.LeakyReLU())

  model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(0.5))

  model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(0.5))

  model.add(layers.Flatten())
  model.add(layers.Dense(1))

  return model

In [5]:
generator = make_generator_model()
generator.summary()
discriminator = make_discriminator_model()
discriminator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16384)             1654784   
                                                                 
 batch_normalization (BatchN  (None, 16384)            65536     
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 16384)             0         
                                                                 
 reshape (Reshape)           (None, 8, 8, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 16, 16, 128)      819200    
 nspose)                                                         
                                                                 
 batch_normalization_1 (Batc  (None, 16, 16, 128)      5

Both generator and discriminator use Adam optimizers with learning rate 0.0002

Last function is Binary Cross Entropy.

In [6]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
generator_optimizer = tf.keras.optimizers.Adam(0.0002)
discriminator_optimizer = tf.keras.optimizers.Adam(0.0002)

def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
  real_loss = cross_entropy(tf.ones_like(real_output), real_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  total_loss = real_loss + fake_loss
  return total_loss

Loading checkpoint. This part is used for continuing training process.

You can change the storing location at checkpoint_dir.

In [7]:
checkpoint_dir = 'drive/MyDrive/train/DCGAN_check'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                  discriminator_optimizer=discriminator_optimizer,
                  generator=generator,
                  discriminator=discriminator)
manager = tf.train.CheckpointManager(checkpoint, checkpoint_prefix, max_to_keep=5)
checkpoint.restore(manager.latest_checkpoint)

Loading Dataset. Those images will be cropped into 128x128 colored images.

Batch size is 256.

The first parameter of image_dataset_from_directory is the location of the dataset folder.

In [8]:
BATCH_SIZE = 256

all_images = tf.keras.preprocessing.image_dataset_from_directory(
    'drive/MyDrive/img_align_celeba/',
    batch_size=1,
    image_size=(218,178),
    shuffle=True,
    labels=None
)

def process(image):
  image = tf.reshape(image, [1, 218, 178, 3])
  image = tf.image.crop_and_resize(image, [[0.14, 0.205, 0.86, 0.795]], [0], [128, 128])
  image = tf.cast((image-127.5) / 127.5 ,tf.float32)
  return image

all_images = all_images.map(process)
all_images = all_images.batch(BATCH_SIZE)

Found 2460 files belonging to 1 classes.


Training process.

It will automatically random;y pick and store the generator and discriminator loss, and also a real and generated image every 10 batchs.

You can change the folder location using the parameter of save_result function.

In [9]:
noise_dim = 100

@tf.function
def train_step(real_images, current_batch_size):
  noise = tf.random.normal([current_batch_size, noise_dim])

  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    generated_images = generator(noise, training=True)

    real_output = discriminator(real_images, training=True)
    fake_output = discriminator(generated_images, training=True)

    gen_loss = generator_loss(fake_output)
    disc_loss = discriminator_loss(real_output, fake_output)
  
  gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
  generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
  gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
  discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

  return gen_loss, disc_loss, real_images, generated_images, fake_output


def train(dataset, epochs, last_epoch):
  for epoch in range(epochs):
    start = time.time()
    recent_epoch = epoch + last_epoch + 1
    data = []
    real_images = []
    generated_images = []
    batch_num = 0
    print('Start training for epoch {}'.format(recent_epoch))

    for images in dataset:
      current_batch_size = images.shape[0]
      images = tf.reshape(images, [current_batch_size, images.shape[2], images.shape[3], images.shape[4]])
      
      gen_loss, disc_loss, real_image, generated_image, fake_output = train_step(images, current_batch_size)
      batch_num += 1
      if (batch_num % 10) == 0:
        r = np.random.randint(current_batch_size)
        data.append((gen_loss.numpy(), disc_loss.numpy(), fake_output.numpy()[r]))
        real_images.append(real_image.numpy()[r])
        generated_images.append(generated_image.numpy()[r])
        print('Batch {} training finished'.format(batch_num))
    
    if (recent_epoch % 5) == 0:
      manager.save()

    save_result(data, real_images, generated_images, recent_epoch, 'drive/MyDrive/train/DCGAN_result')
    display.clear_output(wait=True)
    print ('Time for epoch {} is {} sec'.format(recent_epoch, time.time()-start))
    print ('generator loss:', gen_loss.numpy())
    print ('disciminator loss:', disc_loss.numpy())


def save_result(data, real_images, generated_images, epoch_num, loc):
  wb = xlsxwriter.Workbook(f'{loc}/epoch{epoch_num:03}.xlsx')
  os.makedirs(f'{loc}/epoch{epoch_num:03}/real', exist_ok=True)
  os.makedirs(f'{loc}/epoch{epoch_num:03}/generated', exist_ok=True)
  ws = wb.add_worksheet()
  ws.write_row(0, 0, ('Batch Index', 'Generator Loss', 'Discriminator Loss', 'Generated Image Prediction'))
  batch_num = 1
  for result, real_img, gene_img in zip(data, real_images, generated_images):
    ws.write_row(batch_num, 0, (batch_num, result[0], result[1], result[2]))
    save_img = (real_img * 127.5 + 127.5)
    save_img = PIL.Image.fromarray(np.uint8(save_img))
    save_img.save(f'{loc}/epoch{epoch_num:03}/real/{batch_num:03}.png') 
    save_img = (gene_img * 127.5 + 127.5)
    save_img = PIL.Image.fromarray(np.uint8(save_img))
    save_img.save(f'{loc}/epoch{epoch_num:03}/generated/{batch_num:03}.png')
    batch_num += 1
  wb.close()

In [10]:
train(all_images, 100, 0)

Time for epoch 100 is 15.572921991348267 sec
generator loss: 4.296874
disciminator loss: 0.14261913
